In [ ]:
from brian2 import *

: 

In [ ]:
# Simple Neuron example
start_scope()

tau = 10*ms

eqs = '''
	dv/dt = (1-v)/tau : 1
	'''

G = NeuronGroup(1, eqs)
print(f'before v = {G.v[0]}')
run(100*ms)
print(f'after v = {G.v[0]}')

In [ ]:
print('Expected value of v = %s' % (1-exp(-100*ms/tau)))

In [ ]:
start_scope()
G = NeuronGroup(1, eqs, method='exact')
M = StateMonitor(G, 'v', record=True)

run(30*ms)

plot(M.t/ms, M.v[0])
xlabel('Time (ms)')
ylabel('v')


In [ ]:
start_scope()

G = NeuronGroup(1, eqs, method='exact')
M = StateMonitor(G, 'v', record=0)

run(30*ms)

plot(M.t/ms, M.v[0], 'C0', label='Brian')
plot(M.t/ms, 1-exp(-M.t/tau), 'C1--',label='Analytic')
xlabel('Time (ms)')
ylabel('v')
legend();


In [ ]:
start_scope()

tau = 10*ms
eqs = '''
dv/dt = (sin(2*pi*100*Hz*t)-v)/tau : 1
'''

# Change to Euler Method because exact integrator doesnt work here
G = NeuronGroup(1, eqs, method='euler')
M = StateMonitor(G, 'v', record=0)

G.v = 5 # initial value

run(60*ms)

plot(M.t/ms, M.v[0])
xlabel('Time (ms)')
ylabel('v');

In [ ]:
start_scope()

tau = 10*ms
eqs='''
dv/dt = (1-v)/tau : 1
'''
G = NeuronGroup(1, eqs, threshold='v>0.8', reset='v = 0', method='exact')

M = StateMonitor(G, 'v', record=0)
run(50*ms)

plot(M.t/ms, M.v[0])
xlabel('Time (ms)')
ylabel('v');

In [ ]:
start_scope()
# Start with the default clock (100us per time step)
defaultclock.dt = 0.1*ms

# Define the group of 100 neurons
N = 100  # number of neurons
tau = 10*ms  # time constant
vr = -70*mV  # resting potential
vt = -60*mV  # spiking threshold
El = -70*mV  # reversal potential
eqs = '''
dv/dt = (El - v + I)/tau : volt
I : volt
'''

neurons = NeuronGroup(N, eqs, threshold='v>vt', reset='v=vr', method='euler')
neurons.v = vr  # initial potential
neurons.I = '20*mV * i / N'  # different input current for each neuron

# Record the spike times of each neuron
spikemon = SpikeMonitor(neurons)

# Run the simulation for a certain amount of time
run(100*ms)

# Plot the spike times
plot(spikemon.t/ms, spikemon.i, '.k')
xlabel('Time (ms)')
ylabel('Neuron index')
show()


In [ ]:
morpho = Soma(diameter=30*um)
morpho.axon = Cylinder(length=100*um, diameter=1*um, n=10)
morpho.dendrite = Cylinder(length=50*um, diameter=2*um, n=5)

In [ ]:
morpho.dendrite.branch1 = Cylinder(length=50*um, diameter=1*um, n=3)
morpho.dendrite.branch2 = Cylinder(length=50*um, diameter=1*um, n=3)

In [ ]:
>>> morpho = Soma(diameter=30*um)
>>> morpho.L = Cylinder(length=10*um, diameter=1*um, n=3)
>>> morpho.L1 = Cylinder(length=5*um, diameter=1*um, n=3)
>>> morpho.L2 = Cylinder(length=5*um, diameter=1*um, n=3)
>>> morpho.L3 = Cylinder(length=5*um, diameter=1*um, n=3)
>>> morpho.R = Cylinder(length=10*um, diameter=1*um, n=3)
>>> morpho.RL = Cylinder(length=5*um, diameter=1*um, n=3)
>>> morpho.RR = Cylinder(length=5*um, diameter=1*um, n=3)

In [ ]:
morpho.topology()

In [ ]:
new_morpho = morpho.generate_coordinates()

In [ ]:
new_morpho = morpho.generate_coordinates(section_randomness=25,
                                         compartment_randomness=15)

In [ ]:
start_scope()
set_device('cpp_standalone', build_on_run=False)

tau = 10*ms
I = 1  # input current
eqs = '''
dv/dt = (I-v)/tau : 1
'''
G = NeuronGroup(10, eqs, method='exact')
G.v = 'rand()'
mon = StateMonitor(G, 'v', record=True)
run(20*ms)
I = 0
run(80*ms)
# Actually generate/compile/run the code:
device.build()

plt.plot(mon.t/ms, mon.v.T)
plt.gca().set(xlabel='t (ms)', ylabel='v')
plt.show()

In [ ]:
start_scope()

I_e = 1.5*uA
simulation_time = 100*ms
# neuron RTM parameters
El = -67 * mV
EK = -100 * mV
ENa = 50 * mV
ESyn = 0 * mV
gl = 0.1 * msiemens
gK = 80 * msiemens
gNa = 100 * msiemens

C = 1 * ufarad

weight = 0.25
gSyn = 1.0 * msiemens
tau_d = 2 * ms
tau_r = 0.2 * ms

# forming RTM model with differential equations
eqs = """
alphah = 0.128 * exp(-(vm + 50.0*mV) / (18.0*mV))/ms :Hz
alpham = 0.32/mV * (vm + 54*mV) / (1.0 - exp(-(vm + 54.0*mV) / (4.0*mV)))/ms:Hz
alphan = 0.032/mV * (vm + 52*mV) / (1.0 - exp(-(vm + 52.0*mV) / (5.0*mV)))/ms:Hz

betah  = 4.0 / (1.0 + exp(-(vm + 27.0*mV) / (5.0*mV)))/ms:Hz
betam  = 0.28/mV * (vm + 27.0*mV) / (exp((vm + 27.0*mV) / (5.0*mV)) - 1.0)/ms:Hz
betan  = 0.5 * exp(-(vm + 57.0*mV) / (40.0*mV))/ms:Hz

membrane_Im = I_ext + gNa*m**3*h*(ENa-vm) +
              gl*(El-vm) + gK*n**4*(EK-vm) + gSyn*s_in*(-vm): amp
I_ext : amp
s_in  : 1

dm/dt = alpham*(1-m)-betam*m : 1
dn/dt = alphan*(1-n)-betan*n : 1
dh/dt = alphah*(1-h)-betah*h : 1

ds/dt = 0.5 * (1 + tanh(0.1*vm/mV)) * (1-s)/tau_r - s/tau_d : 1

dvm/dt = membrane_Im/C : volt
"""

neuron = NeuronGroup(2, eqs, method="exponential_euler")

# initialize variables
neuron.vm = [-70.0, -65.0]*mV
neuron.m = "alpham / (alpham + betam)"
neuron.h = "alphah / (alphah + betah)"
neuron.n = "alphan / (alphan + betan)"
neuron.I_ext = [I_e, 0.0*uA]

S = Synapses(neuron,
             neuron,
             's_in_post = weight*s_pre:1 (summed)')
S.connect(i=0, j=1)

# tracking variables
st_mon = StateMonitor(neuron, ["vm", "s", "s_in"], record=[0, 1])

# running the simulation
run(simulation_time)

# plot the results
fig, ax = plt.subplots(2, figsize=(10, 6), sharex=True,
                       gridspec_kw={'height_ratios': (3, 1)})

ax[0].plot(st_mon.t/ms, st_mon.vm[0]/mV,
           lw=2, c="r", alpha=0.5, label="neuron 0")
ax[0].plot(st_mon.t/ms, st_mon.vm[1]/mV,
           lw=2, c="b", alpha=0.5, label='neuron 1')
ax[1].plot(st_mon.t/ms, st_mon.s[0],
           lw=2, c="r", alpha=0.5, label='s, neuron 0')
ax[1].plot(st_mon.t/ms, st_mon.s_in[1],
           lw=2, c="b", alpha=0.5, label='s_in, neuron 1')
ax[0].set(ylabel='v [mV]', xlim=(0, np.max(st_mon.t / ms)),
          ylim=(-100, 50))
ax[1].set(xlabel="t [ms]", ylabel="s", ylim=(0, 1))

ax[0].legend()
ax[1].legend()

plt.show()

: 